In [1]:
import datetime
import numpy as np
import pandas as pd
from pandas import Series
from itertools import groupby
import matplotlib.pyplot as plt

%matplotlib inline  

In [156]:
df = pd.read_csv('../../data/train.csv')
ids = df['KPI ID'].unique()

In [3]:
df.head()

,timestamp,value,label,KPI ID
0,1493568000,1.901639,0,02e99bd4f6cfb33f
1,1493568060,1.786885,0,02e99bd4f6cfb33f
2,1493568120,2.000000,0,02e99bd4f6cfb33f
3,1493568180,1.885246,0,02e99bd4f6cfb33f
4,1493568240,1.819672,0,02e99bd4f6cfb33f


In [4]:
df['label'].value_counts()

0    2422815
1      53500
Name: label, dtype: int64

In [5]:
df['value'].describe()

count    2.476315e+06
mean     1.080211e+08
std      3.196926e+08
min     -4.546069e+00
25%      3.700000e+00
50%      1.293438e+03
75%      2.523000e+03
max      1.608813e+09
Name: value, dtype: float64

In [157]:
# upsampling
dfs = []
for _id in ids:
    df_tmp = df[df['KPI ID'] == _id]
    df_tmp['datetime'] = pd.to_datetime(df_tmp['timestamp'], unit='s')
    df_tmp = df_tmp.set_index('datetime')
    df_tmp = df_tmp.sort_index()
    df_tmp = df_tmp.resample('1T').ffill()
    
    df_tmp['id'] = df_tmp['KPI ID'] + '-' + df_tmp.index.astype('str') 
    df_tmp = df_tmp.set_index('id')
    
    dfs.append(df_tmp)
    print(f'Done with {_id}')
    
df_sampled = pd.concat(dfs)

/Users/alex/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Done with 02e99bd4f6cfb33f
Done with 9bd90500bfd11edb
Done with da403e4e3f87c9e0
Done with a5bf5d65261d859a
Done with 18fbb1d5a5dc099d
Done with 09513ae3e75778a3
Done with c58bfcbacb2822d1
Done with 1c35dbf57f55f5e4
Done with 046ec29ddf80d62e
Done with 07927a9a18fa19ae
Done with 54e8a140f6237526
Done with b3b2e6d1a791d63a
Done with 8a20c229e9860d0c
Done with 769894baefea4e9e
Done with 76f4550c43334374
Done with e0770391decc44ce
Done with 8c892e5525f3e491
Done with 40e25005ff8992bd
Done with cff6d3c01e6a6bfa
Done with 71595dd7171f4540
Done with 7c189dd36f048a6c
Done with a40b1df87e3f1c87
Done with 8bef9af9a922e0b3
Done with affb01ca2b4f0b45
Done with 9ee5879409dccef9
Done with 88cf3a776ba00e7c


In [188]:
df_sampled[df_sampled['KPI ID'] == ids[1]].describe()

,timestamp,value,label
count,1.317950e+05,131795.000000,131795.000000
mean,1.497521e+09,31.687321,0.001085
std,2.283075e+06,11.790595,0.032922
min,1.493568e+09,5.333333,0.000000
25%,1.495545e+09,22.916666,0.000000
50%,1.497522e+09,30.076923,0.000000
75%,1.499499e+09,38.099998,0.000000
max,1.501476e+09,91.307693,1.000000


In [189]:
# normalization
for _id in ids:
    df_tmp = df_sampled[df_sampled['KPI ID'] == _id]['value']
    df_sampled.loc[df_sampled['KPI ID'] == _id, 'value'] = (df_tmp - df_tmp.mean()) / df_tmp.std()

In [191]:
df_sampled[df_sampled['KPI ID'] == ids[2]].describe()

,timestamp,value,label
count,1.317900e+05,1.317900e+05,131790.000000
mean,1.497521e+09,6.071169e-14,0.058183
std,2.283031e+06,1.000000e+00,0.234091
min,1.493568e+09,-2.146061e+00,0.000000
25%,1.495545e+09,-4.766835e-01,0.000000
50%,1.497522e+09,-1.863569e-01,0.000000
75%,1.499499e+09,5.394593e-01,0.000000
max,1.501475e+09,1.462030e+01,1.000000


In [193]:
df_sampled.to_csv('../../data/train_sampled_normalized.csv')